In [ ]:
!pip install -U duckduckgo_search
# Doc: https://pypi.org/project/duckduckgo-search/

In [ ]:
!pip install newspaper3k

In [ ]:
!pip install mechanize

In [2]:
from duckduckgo_search import DDGS
from newspaper import Article
import json

In [3]:
from duckduckgo_search import ddg

In [ ]:
with DDGS() as ddgs:
    keywords = '"huracán otis"'
    ddgs_news_gen = ddgs.text(
      keywords,
      region="mx-es",
      safesearch="off",
      backend='lite',
      timelimit='m',
      max_results=500
    )
    
urls = []  # Inicializamos una lista vacía para almacenar las URLs
for r in ddgs_news_gen:
    urls.append(r['href'])

In [21]:
results = ddg('"huracán otis"', max_results=100)
urls = [result['href'] for result in results]

In [22]:
urls

['https://www.bbc.com/mundo/articles/c807nr77p4xo',
 'https://es.wikipedia.org/wiki/Huracán_Otis',
 'https://www.bbc.com/mundo/articles/cjl03ep04yyo',
 'https://ciencia.nasa.gov/ciencias-terrestres/el-huracan-otis/',
 'https://www.cnn.com/videos/spanish/2023/10/25/hurcan-otis-acapulco-orix.cnn',
 'https://www.cnn.com/videos/spanish/2023/10/25/otis-huracan-acapulco-guerrero-perspectivas-mexico-tv.cnn',
 'https://cnnespanol.cnn.com/2023/10/24/tormenta-tropical-otis-fuerza-huracan-sur-mexico-trax/',
 'https://www.bloomberg.com/news/articles/2023-10-25/huracan-otis-azota-mexico-tras-tocar-tierra-cerca-de-acapulco',
 'https://edition.cnn.com/videos/spanish/2023/10/31/victimas-huracan-otis-entierro-cuerpos-acapulco-conclusiones-tv.cnn',
 'https://cnnespanol.cnn.com/2023/10/27/huracan-otis-danos-noticias-acapulco-mexico-orix-2/',
 'https://www.latimes.com/espanol/mexico/articulo/2023-10-29/victimas-por-paso-del-huracan-otis-se-elevan-a-43-en-el-sur-de-mexico-y-los-desaparecidos-suben-a-36',
 

In [33]:
import mechanize
import re
import time

def extract_urls(query, num_urls=100):
    # Configurar el navegador
    br = mechanize.Browser()
    br.set_handle_robots(False)  # Ignorar el archivo robots.txt
    br.addheaders = [('User-agent', 'Mozilla/5.0')]  # Definir el User-agent

    # Abrir DuckDuckGo con parámetros para español (México)
    base_url = "https://html.duckduckgo.com/html/?kl=mx-es"
    br.open(base_url)

    # Seleccionar el formulario de búsqueda y enviar la consulta
    br.select_form(nr=0)
    br["q"] = query
    response = br.submit()

    urls = set()

    # Regex para extraer URLs
    link_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

    while len(urls) < num_urls:
        # Extraer URLs de la página actual
        for link in br.links():
            match = link_pattern.match(link.url)
            if match and link.url not in urls:  # Verifica si la URL no está ya en el conjunto
                urls.add(link.url)
                print(link.url)  # Imprime la URL recién agregada
                if len(urls) == num_urls:
                    return list(urls)
        
        # Intentar enviar el formulario "Next" para la próxima página de resultados
        try:
            br.select_form(nr=1)
            br.submit()
        except (mechanize.FormNotFoundError, IndexError):
            break

        time.sleep(2)

    return list(urls)

# Prueba
#urls = extract_urls('"El huracán Otis"')

urls = extract_urls('huracán otis acapulco"')


https://www.bbc.com/mundo/articles/c807nr77p4xo
https://politica.expansion.mx/mexico/2023/11/01/huracan-otis-en-acapulco-una-semana-de-caos-y-angustia
https://www.nytimes.com/es/2023/10/30/espanol/acapulco-huracan-otis-refugiados.html
https://politica.expansion.mx/estados/2023/10/25/huracan-otis-danos-acapulco
https://www.forbes.com.mx/acapulco-lo-que-el-huracan-otis-nos-dice-del-cambio-climatico/
https://apnews.com/climate-and-environment/general-news-cf58d435f66705c3584124bab3325293
https://mvsnoticias.com/nacional/estados/2023/10/25/huracan-otis-deja-graves-danos-asi-fue-como-toco-tierra-en-acapulco-videos-611879.html
https://www.am.com.mx/nacional/2023/10/25/huracan-otis-devasta-al-hotel-princess-de-acapulco-681179.html
https://www.bbc.com/mundo/articles/cxw9739pyvjo
https://heraldodemexico.com.mx/nacional/2023/10/25/la-devastacion-de-otis-en-imagenes-desde-acapulco-asi-quedo-destruido-549540.html
https://elpais.com/mexico/2023-10-28/el-acapulco-arrasado-despues-de-otis.html
https:

In [ ]:
for url in results:
    print(url)

In [ ]:
urls

In [25]:
def extract_content_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "content": article.text,
            "url": url,
            "date": article.publish_date.strftime('%Y-%m-%d') if article.publish_date else None,
            "image_url": article.top_image
        }
    except Exception as e:
        print(f"Error al extraer contenido de {url}. Error: {e}")
        return None

In [34]:
import os

# Definimos la ruta del archivo
archivo_path = './02_news/noticias_huracan_otis.jsonl'

# Diccionario para almacenar las noticias, usando la URL como llave
noticias_dict = {}

# Si el archivo ya existe, lo leemos para cargar las noticias existentes
if os.path.exists(archivo_path):
    with open(archivo_path, 'r', encoding='utf-8') as file:
        for line in file:
            noticia = json.loads(line.strip())
            noticias_dict[noticia['url']] = noticia

# Procesamos las URLs nuevas
for r in urls:
    print(r)
    if r not in noticias_dict:  # Verificamos que la URL no exista ya en el diccionario
        content = extract_content_from_url(r)
        if content:
            noticias_dict[content['url']] = content  # Agregamos la noticia al diccionario

# Guardamos las noticias (tanto las antiguas como las nuevas) en el archivo
with open(archivo_path, 'w', encoding='utf-8') as file:
    for noticia in noticias_dict.values():
        file.write(json.dumps(noticia, ensure_ascii=False) + '\n')

print("¡Proceso completado!")


https://elpais.com/mexico/2023-10-27/acapulco-el-antes-y-despues-de-otis.html
https://elpais.com/mexico/2023-10-25/el-huracan-otis-en-imagenes.html
https://elpais.com/mexico/2023-10-26/acapulco-la-tragedia.html
https://www.bbc.com/mundo/articles/c72rg2g5lyeo
https://www.telediario.mx/nacional/huracan-otis-golpea-acapulco-guerrero-fotos-videos
Error al extraer contenido de https://www.telediario.mx/nacional/huracan-otis-golpea-acapulco-guerrero-fotos-videos. Error: Article `download()` failed with HTTPSConnectionPool(host='www.telediario.mx', port=443): Read timed out. (read timeout=7) on URL https://www.telediario.mx/nacional/huracan-otis-golpea-acapulco-guerrero-fotos-videos
https://www.msn.com/es-mx/dinero/noticias/hurac%C3%A1n-otis-en-acapulco-sube-a-47-la-cifra-de-muertos/ar-AA1jo7I3
https://apnews.com/climate-and-environment/general-news-cf58d435f66705c3584124bab3325293
https://cnnespanol.cnn.com/2023/10/24/tormenta-tropical-otis-fuerza-huracan-sur-mexico-trax/
https://www.poresto